In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, train_test_split

In [31]:
df = pd.read_csv(r"data\labelled\santacruz_labelled_days.csv")

In [32]:
df

,DATE,DPT,WBT,DBT,Normal_Temp,Heatwave,Heatwave_Days
0,2010-01-01,16.0875,19.775,25.725000,30.4,0.0,0
1,2010-01-02,18.2000,21.375,26.975000,30.4,0.0,0
2,2010-01-03,19.7375,22.025,26.300000,30.4,0.0,0
3,2010-01-04,18.0250,20.450,24.575000,30.4,0.0,0
4,2010-01-05,18.5500,20.575,24.175000,30.4,0.0,0
...,...,...,...,...,...,...,...
5474,2024-12-27,13.4625,27.400,21.847186,31.9,0.0,0
5475,2024-12-28,13.4625,27.400,21.261927,31.9,0.0,0
5476,2024-12-29,13.4625,27.400,21.481941,31.9,0.0,0
5477,2024-12-30,13.4625,27.400,21.161551,31.9,0.0,0


In [33]:
df.isnull().sum()

DATE             0
DPT              0
WBT              0
DBT              0
Normal_Temp      0
Heatwave         0
Heatwave_Days    0
dtype: int64

In [34]:
heatwave_counts = df['Heatwave_Days'].value_counts()
heatwave_counts

Heatwave_Days
0    5417
1      62
Name: count, dtype: int64

In [42]:
target_col = ['Heatwave_Days']

In [43]:
features = ['DPT', 'WBT', 'DBT', 'Normal_Temp']
target = ['Heatwave_Days']

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [44]:
# Convert to sequences for LSTM
sequence_length = 10
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i+sequence_length].values)
    y.append(df[target].iloc[i+sequence_length])
X, y = np.array(X), np.array(y)

In [45]:
y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [46]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_idx, test_idx in kfold.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [47]:
model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, len(features))),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(50, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

c:\Users\LaukikS\anaconda3\envs\dm-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [49]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9800 - loss: 0.2620 - val_accuracy: 0.9890 - val_loss: 0.0602
Epoch 2/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9882 - loss: 0.0768 - val_accuracy: 0.9890 - val_loss: 0.0631
Epoch 3/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9881 - loss: 0.0724 - val_accuracy: 0.9890 - val_loss: 0.0648
Epoch 4/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9900 - loss: 0.0615 - val_accuracy: 0.9890 - val_loss: 0.0607
Epoch 5/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9891 - loss: 0.0664 - val_accuracy: 0.9890 - val_loss: 0.0608
Epoch 6/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9872 - loss: 0.0730 - val_accuracy: 0.9890 - val_loss: 0.0616
Epoch 7/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9874 - loss: 0.0761 - val_accuracy: 0.9890 - val_loss: 0.0606
Epoch 8/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9891 - loss: 0.0654 - val_accur

In [50]:
loss, accuracy = model.evaluate(X_test, y_test)
accuracies.append(accuracy)
print(f"Fold {fold_no} Accuracy: {accuracy:.4f}")
fold_no += 1

# Print average accuracy
print(f"Average Cross-Validation Accuracy: {np.mean(accuracies):.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9886 - loss: 0.0582
Fold 1 Accuracy: 0.9890
Average Cross-Validation Accuracy: 0.9890
